# 2. Extraction des motifs frequents

In [1]:
 
%matplotlib inline

import math
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.tree import DecisionTreeRegressor
from itertools import combinations

In [2]:
def chargerData(path):
    data = pd.read_csv(path)
    return data
data1 = chargerData('Dataset3.csv')

# Preprocessing

In [3]:
#convert the data 
#temperature to float
#replace the , to .
data1['Temperature'] = data1['Temperature'].str.replace(',','.')
data1['Humidity'] = data1['Humidity'].str.replace(',','.')
data1['Rainfall'] = data1['Rainfall'].str.replace(',','.')

#humidity to float
data1['Temperature'] = data1['Temperature'].astype(float)
data1['Humidity'] = data1['Humidity'].astype(float)
data1['Rainfall'] = data1['Rainfall'].astype(float)

In [4]:
def replace_missing_values(data, col_name):
    data[col_name] = data[col_name].fillna(data[col_name].mean())
    return data

replace_missing_values(data1, 'Temperature')
replace_missing_values(data1, 'Humidity')
replace_missing_values(data1, 'Rainfall')

,Temperature,Humidity,Rainfall,Soil,Crop,Fertilizer
0,24.87,82.84,295.61,Clayey,rice,DAP
1,28.69,96.65,178.96,laterite,Coconut,Good NPK
2,20.27,81.64,270.44,silty clay,rice,MOP
3,25.07,95.02,192.90,sandy,Coconut,Urea
4,25.04,95.90,174.80,coastal,Coconut,Urea
...,...,...,...,...,...,...
290,26.07,96.70,143.53,sandy,Coconut,MOP
291,25.63,83.53,209.90,silty clay,rice,MOP
292,24.52,83.54,230.45,Clayey,rice,MOP
293,25.66,83.47,217.38,Clayey,rice,MOP


## discrétisation

### 1. discrétisation en classe effectif égale

In [5]:
def equal_frequency(data, data_col,label):
    num_classes = int((1 + 10 / 3) * math.log(len(data[data_col]), 10))

    # Perform equal-frequency discretization
    discretized = pd.qcut(data[data_col], q=num_classes, labels=False, duplicates="drop")
    

    # Get the bin edges
    _, bin_edges = pd.qcut(data[data_col], q=num_classes, retbins=True, duplicates="drop")

    # Create custom labels based on the bin edges
    labels = [f"{label}class{i}" for i in range(len(bin_edges)-1)]
    # labels = [f"[{bin_edges[i]:.1f}, {bin_edges[i+1]:.1f}]" for i in range(len(bin_edges)-1)]

    # Assign the custom labels to the discretized column
    discretized_with_labels = discretized.map(dict(enumerate(labels)))

    return discretized_with_labels


def equal_width(data,data_col,label):
    num_classes = int((1 + 10 / 3) * math.log(len(data[data_col]), 10))
    # Perform equal-width discretization
    discretized = pd.cut(data[data_col], bins=num_classes, labels=False, duplicates="drop")

    # Get the bin edges
    _, bin_edges = pd.cut(data[data_col], bins=num_classes, retbins=True, duplicates="drop")

    # Create custom labels based on the bin edges
    labels = [f"{label}class{i}" for i in range(len(bin_edges)-1)]
    print(labels)
    # labels = [f"[{bin_edges[i]:.1f}, {bin_edges[i+1]:.1f}]" for i in range(len(bin_edges)-1)]

    # Assign the custom labels to the discretized column
    discretized_with_labels = discretized.map(dict(enumerate(labels)))

    return discretized_with_labels 

In [6]:
equal_frequency(data1,'Temperature','Temp')

0      Tempclass3
1      Tempclass8
2      Tempclass0
3      Tempclass3
4      Tempclass3
          ...    
290    Tempclass5
291    Tempclass4
292    Tempclass3
293    Tempclass4
294    Tempclass2
Name: Temperature, Length: 295, dtype: object

In [7]:
data1['Temperature']=equal_frequency(data1,'Temperature','Temp')
data1['Humidity']=equal_frequency(data1,'Humidity','Hum')
data1['Rainfall']=equal_frequency(data1,'Rainfall','Rain')

In [8]:
data1

,Temperature,Humidity,Rainfall,Soil,Crop,Fertilizer
0,Tempclass3,Humclass2,Rainclass9,Clayey,rice,DAP
1,Tempclass8,Humclass8,Rainclass2,laterite,Coconut,Good NPK
2,Tempclass0,Humclass1,Rainclass9,silty clay,rice,MOP
3,Tempclass3,Humclass7,Rainclass3,sandy,Coconut,Urea
4,Tempclass3,Humclass8,Rainclass2,coastal,Coconut,Urea
...,...,...,...,...,...,...
290,Tempclass5,Humclass8,Rainclass0,sandy,Coconut,MOP
291,Tempclass4,Humclass3,Rainclass5,silty clay,rice,MOP
292,Tempclass3,Humclass3,Rainclass7,Clayey,rice,MOP
293,Tempclass4,Humclass3,Rainclass6,Clayey,rice,MOP


## Apriori


In [9]:
def support(data, itemset):
    return len(data[data.apply(lambda row: all(item in row.values for item in itemset), axis=1)])
support(data1,('Tempclass3',))

30

In [10]:
def create_candidats(items,n):

    uples = list(combinations(items, n))
    return uples
def l(dataset2_bis, candidatas, supp_min):
    l = []
    if support(dataset2_bis, candidatas) >= supp_min:
        l.append(candidatas)
    return l

In [11]:
def appriori(data, itemset, suppmin):
    #start by the C1
    l = []
    suppmin = int(len(data)*suppmin)
    stop = False
    n = 1
    candidats = create_candidats(itemset,n)
    while(not stop):
        supps = 0
        new_items = []
        for c in candidats:
            if support(data,c) >= suppmin:
                supps += 1
                l.append(c)
            
                new_items.append(c)
        items  = np.unique(np.array([item for tpl in new_items for item in tpl]))
        candidats = []
        candidats = create_candidats(items,n+1)
        if supps == 0:
            stop = True
        else:
            n += 1
    return l
def get_best_rules(data,l, confmin):
    res = {}
    for x in l:
        res[x] =[]
        rules = regle_association(data,x)
        for r in rules:
            if confiance(data,r)>=confmin:
                res[x].append(r)
        if len(res[x])==0:
            #remove it
            res.pop(x)
    return res

def regle_association(dataset, combination):
    length = len(combination)
    regles = []
    for i in range(1, length):
        combi = list(combinations(combination, i))
        for c in combi:
            c = list(c)
            c2 = [x for x in combination if x not in c]
            regles.append((c, c2))
    return regles
def confiance(dataset,regle):
    return (support(dataset,(regle[0]+regle[1])))/support(dataset,regle[0])

In [12]:
def get_itemsets(data):
    #get all the unique items from all the attributes
    items = np.unique(np.array([item for tpl in data.values for item in tpl]))
    return items

In [14]:
#varying several supp min
for s in range(1,10):
    print("supp min : ",s/10)
    l = appriori(data1,get_itemsets(data1),s/10)
    #print l
    print(l)
    for c in range(1,10):
        print('confiance min: ',c/10)
        res = get_best_rules(data1,l,c/10)
        for r in res:
            # print(r ,": ")
            for x in res[r]:
                print('\t',x[0],'->',x[1])

supp min :  0.1
[('Clayey',), ('Coconut',), ('DAP',), ('Good NPK',), ('Humclass0',), ('Humclass1',), ('Humclass2',), ('Humclass3',), ('Humclass4',), ('Humclass6',), ('Humclass7',), ('Humclass8',), ('Humclass9',), ('MOP',), ('Rainclass0',), ('Rainclass1',), ('Rainclass2',), ('Rainclass3',), ('Rainclass4',), ('Rainclass5',), ('Rainclass6',), ('Rainclass7',), ('Rainclass8',), ('Rainclass9',), ('Tempclass0',), ('Tempclass1',), ('Tempclass2',), ('Tempclass3',), ('Tempclass4',), ('Tempclass5',), ('Tempclass6',), ('Tempclass7',), ('Tempclass8',), ('Tempclass9',), ('Urea',), ('coastal',), ('rice',), ('sandy',), ('silty clay',), ('Clayey', 'rice'), ('Coconut', 'DAP'), ('Coconut', 'Humclass7'), ('Coconut', 'Humclass9'), ('Coconut', 'MOP'), ('Coconut', 'Rainclass0'), ('Coconut', 'Rainclass1'), ('Coconut', 'Rainclass2'), ('Coconut', 'Urea'), ('Coconut', 'coastal'), ('Coconut', 'sandy'), ('DAP', 'rice'), ('Humclass3', 'rice'), ('Humclass4', 'rice'), ('MOP', 'rice'), ('Rainclass8', 'rice'), ('Raincl